# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
# print(os.getcwd())

# Get the current folder and subfolder event data
datadir = os.getcwd() + '/event_data'

# The build-up of file_path_list had to be revamped as it was disturbed by the jupyter snapshots
# Get the file path for all csv's in the subfolder referenced to in datadir
file_path_list=[]
for filename in os.listdir(datadir):
    f=os.path.join(datadir,filename)
    if os.path.isfile(f) & (f[-4:]=='.csv'):
        file_path_list.append(f)


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print('Across all files, there are '+str(len(full_data_rows_list))+' rows')
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

# This is a counter to identify the rows that will be discarded because of a missing value in the first field
rowzero_idx=0

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            rowzero_idx=rowzero_idx+1
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

# Checking on the total of rows discarded
print('Because the artist info was missing (i.e. user login info), we discarded '+str(rowzero_idx)+' rows')

Across all files, there are 8056 rows
Because the artist info was missing (i.e. user login info), we discarded 1236 rows


#### check on total of remaining rows

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(str(sum(1 for line in f))+' have been kept finally')

6821 have been kept finally


# Part II. Apache Cassandra coding portion of the project. 

## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## This is the Apache Cassandra admin code where we create the resources needed

#### Creating a Cluster

In [5]:
# This makes a connection to a Cassandra instance on the local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, we need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# creating a keyspace (equiv to a schema in RDBMS)
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION =
    {'class':'SimpleStrategy','replication_factor':1}""")
except Exception as e:
    print('an error occured:')
    print(e)

#### Set Keyspace

In [7]:
# we instruct the connector that we work on the keyspace that we have just created
try:
    session.set_keyspace('udacity')
except Exception as e:
    print('an error occured:')
    print(e)

### Now we need to create tables to run the following queries. With Apache Cassandra we model the database tables on the queries we want to run.

# Queries and tables creation

### Create queries to ask the following three questions of the data

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# We will stick to the NoSQL paradigm and create one table per query
# artist_song_session for query1
# song_playlist_session for query2
# users_song for query3

# Some panda preparation for further analysis on unicity

In [9]:
df=pd.read_csv('event_datafile_new.csv')

In [10]:
df.columns

Index(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
       'level', 'location', 'sessionId', 'song', 'userId'],
      dtype='object')

---

# table1 - artist_song_session for query1

In [11]:
# first, we need to make sure that the columns we consider for the primary key do not come with duplicate values
dfcount1=df.groupby(['sessionId','itemInSession']).size().reset_index().rename(columns={0:'count'})
len(dfcount1[dfcount1['count']>1]) # should equal to 0

0

### query1 formulation

This query will need to be filtered on session_id and item_in_session.

> These 2 fields will need to be included in the PRIMARY KEY as partition key

The results don't need to be ordered in a specific way

> There are thus no other fields to be included in the PRIMARY KEY as clustering columns

In [1]:
query1="SELECT artist,song_title,song_length FROM artist_song_session WHERE session_id=338 and item_in_session=4"

### table1 - artist_song_session CREATE

In [13]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

#table1 needs to be filtered on session_id and item_in_session
#table1 does not need to be ordered in a specific way
try:
    query="""CREATE TABLE IF NOT EXISTS artist_song_session"""
    session.execute(query+"""(session_id int,\
                                item_in_session int,\
                                artist varchar,\
                                song_title varchar,\
                                song_length float,\
                                PRIMARY KEY((session_id),item_in_session))""")
except Exception as e:
    print('an error occured:')
    print(e)

### table1 - artist_song_session data INSERT

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_query1 = "INSERT INTO artist_song_session(session_id,item_in_session,artist,song_title,song_length)"
        insert_query1 = insert_query1 + "VALUES (%s,%s,%s,%s,%s)"
        try:
            session.execute(insert_query1,(int(line[8]),int(line[3]),line[0],line[9],float(line[5])))
        except Exception as e:
            print('an error occured:')
            print(e)

### table1 - artist_song_session data SELECT

In [15]:
## SELECT statement to verify the data was entered into the table
try:
    rows = session.execute(query1)
except Exception as e:
    print('an error occured:')
    print(e)
    
for row in rows:
    print(str(row.artist)+", "+str(row.song_title)+", "+str(row.song_length))

Faithless, Music Matters (Mark Knight Dub), 495.30731201171875


***

# table2 - song_playlist_session for query2

In [17]:
# first, we need to make sure that the columns we consider for the primary key do not come with duplicate values
dfcount2=df.groupby(['userId','sessionId','itemInSession']).size().reset_index().rename(columns={0:'count'})
len(dfcount1[dfcount1['count']>1]) # should equal to 0

0

### query2 formulation

This query will need to be filtered on user_id and session_id.

> These 2 fields will need to be included in the PRIMARY KEY as a partition key

The results need to be ordered by item_in_session

> This 3rd field will thus have to be included in the PRIMARY KEY as a clustering column

In [18]:
query2="SELECT artist,song_title, usr_first_name, usr_last_name FROM song_playlist_session WHERE user_id=10 and session_id=182"


### table2 - song_playlist_session CREATE

In [19]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
try:
    create_query2="""CREATE TABLE IF NOT EXISTS song_playlist_session"""
    session.execute(create_query2+"""(user_id int,\
                                        session_id int,\
                                        item_in_session int,\
                                        artist varchar,\
                                        song_title varchar,\
                                        usr_first_name varchar,\
                                        usr_last_name varchar,\
                                        PRIMARY KEY((user_id,session_id),item_in_session))""")
except Exception as e:
    print(e)

### table2 - song_playlist_session data INSERT

In [20]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_query2 = "INSERT INTO song_playlist_session(user_id,session_id, item_in_session,artist,song_title,usr_first_name,usr_last_name)"
        insert_query2 = insert_query2 + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        try:
            session.execute(insert_query2,(int(line[10]),int(line[8]),int(line[3]),line[0],line[9],line[1],line[4]))
        except Exception as e:
            print('an error occured:')
            print(e)

### table2 - song_playlist_session data SELECT

In [21]:
try:
    rows = session.execute(query2)
except Exception as e:
    print('an error occured:')
    print(e)
    
for row in rows:
    #print(row)
    print(str(row.artist)+", "+str(row.song_title)+", "+str(row.usr_first_name)+", "+str(row.usr_last_name))

Down To The Bone, Keep On Keepin' On, Sylvie, Cruz
Three Drives, Greece 2000, Sylvie, Cruz
Sebastien Tellier, Kilometer, Sylvie, Cruz
Lonnie Gordon, Catch You Baby (Steve Pitron & Max Sanna Radio Edit), Sylvie, Cruz


---

# table3 - users_song for query3

In [24]:
# Here, we don't need to make sure we won't miss on duplicates as we are not interested in the count of listenings of each song by each user


### query3 formulation

This query will need to be filtered on song_title.

> This field will need to be included in the PRIMARY KEY as partition key

The results don't need to be ordered in a specific way

> There are thus no other fields to be included in the PRIMARY KEY as clustering columns for ordering reasons

We need to allow for multiple records referring to the same song, but listened by multiple users.

> The field user_id will have to be included in the PRIMARY KEY as clustering column

In [30]:
query3="SELECT usr_first_name, usr_last_name FROM users_song WHERE song_title='All Hands Against His Own'"


### table3 - users_song CREATE

In [31]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
try:
    create_query3="""CREATE TABLE IF NOT EXISTS users_song"""
    session.execute(create_query3+"""(song_title varchar,\
                                        user_id int,\
                                        usr_first_name varchar,\
                                        usr_last_name varchar,\
                                        PRIMARY KEY (song_title,user_id))""")
except Exception as e:
    print('an error occured')
    print(e)

                    

### table3 - users_song data INSERT

In [33]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_query2 = "INSERT INTO users_song(song_title,user_id,usr_first_name,usr_last_name)"
        insert_query2 = insert_query2 + "VALUES (%s,%s,%s,%s)"
        try:
            session.execute(insert_query2,(line[9],int(line[10]),line[1],line[4]))
        except Exception as e:
            print('an error occured:')
            print(e)

### table3 - users_song data SELECT

In [34]:
try:
    rows=session.execute(query3)
except Exception as e:
    print('an error occured')
    print(e)
    
for row in rows:
    print(row.usr_first_name+" "+row.usr_last_name)
    

Jacqueline Lynch
Tegan Levine
Sara Johnson


---

### Drop the tables before closing out the sessions

In [29]:
try:
    session.execute('DROP TABLE artist_song_session')
except Exception as e:
    print(e)
    
try:
    session.execute('DROP TABLE song_playlist_session')
except Exception as e:
    print(e)
    
try:
    session.execute('DROP TABLE users_song')
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [95]:
session.shutdown()
cluster.shutdown()